## Mask Generation Inference using Online Endpoints

This sample shows how to deploy `mask-generation` type models to an online endpoint for inference.

### Task
`mask-generation` takes in images and prompts (input points, input_boxes, input_labels) and for each image, generates segmentation masks based on the prompts given.

### Model
Models that can perform the `mask-generation` task are tagged with `mask-generation`. We will use the `facebook-sam-vit-huge` model in this notebook. If you opened this notebook from a specific model card, remember to replace the specific model name.

### Inference data
We will use the [odFridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip) dataset.


### Outline
1. Setup pre-requisites
2. Pick a model to deploy
3. Prepare data for inference
4. Deploy the model to an online endpoint for real time inference
5. Test the endpoint
6. Clean up resources - delete the online endpoint

### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Connect to `azureml` system registry

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace_name = "<WORKSPACE_NAME>"
workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

# The models are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)

### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `mask-generation` task. In this example, we use the `facebook-sam-vit-huge` model. If you have opened this notebook for a different model, replace the model name accordingly.

In [ ]:
model_name = "facebook-sam-vit-huge"

foundation_model = registry_ml_client.models.get(name=model_name, label="latest")
print(
    f"\n\nUsing model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for inferencing"
)

### 3. Prepare data for inference

We will use the [odFridgeObjects](https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip) dataset for this mask-generation task.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# Create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# Download data
download_url = "https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-object-detection/odFridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# Extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")
# Delete zip file
os.remove(data_file)

In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "images", "99.jpg")
Image(filename=sample_image)

### 4. Deploy the model to an online endpoint for real time inference
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import time
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
)

# Endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
timestamp = int(time.time())
online_endpoint_name = "mask-gen-" + str(timestamp)
# Create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for "
    + foundation_model.name
    + ", for mask-generation task",
    # auth_mode="key",
)
workspace_ml_client.begin_create_or_update(endpoint).wait()

In [ ]:
from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

deployment_name = "mask-gen-mlflow-deploy"

# Create a deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=foundation_model.id,
    instance_type="Standard_DS5_V2",  # Use GPU instance type like Standard_NC6s_v3 for faster inference
    instance_count=1,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=1,
        request_timeout_ms=90000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=10,
    ),
)
workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 5. Test the endpoint

We will fetch some sample data from the test dataset and submit to online endpoint for inference.

#### Input Data Parameters

- **`image`**: 
  - Column in the DataFrame containing images as base64-encoded strings or URLs.

- **`input_points`**: 
  - String representation of a numpy array with shape `(point_batch_size, num_points, 2)`.
  - Represents 2D spatial input points for prompt encoding.
  - Organized as a list of lists, converted into tensors with dimensions for point batches (i.e. how many segmentation masks do we want the model to predict per input point), number of points, and x, y coordinates.
  - Padding points are represented as `(0, 0)` and are excluded from embedding computations.

- **`input_boxes`**: 
  - String representation of a numpy array with shape `(num_boxes, 4)`.
  - Encodes input boxes for points to enhance mask generation.
  - Structured as a nested list that translates to tensors indicating number of boxes per image, and coordinates for each box's top left (`x1`, `y1`) and bottom right (`x2`, `y2`) points.

- **`input_labels`**: 
  - String representation of a numpy array with shape `(point_batch_size, num_points)`.
  - Includes labels for the points utilized by the prompt encoder.
  - Label types are `1` for object points, `0` for non-object points, `-1` for background points, and `-10` for padding points.

- **`multimask_output`**: 
  - Boolean indicating if multiple masks per input point (`True`) or a single mask (`False`) should be returned.
  - Default is set to `True`, allowing for multiple masks per point unless otherwise specified.


In [ ]:
import base64
import json

sample_image = os.path.join(dataset_dir, "images", "99.jpg")


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


request_json = {
    "input_data": {
        "columns": [
            "image",
            "input_points",
            "input_boxes",
            "input_labels",
            "multimask_output",
        ],
        "index": [0, 1, 2, 3, 4],
        "data": [
            # segmentation mask per input point
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                "[[[280,320]], [[300,350]]]",
                "",
                "",
                True,
            ],
            # single segmentation mask for multiple input points
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                "[[[280,320], [300,350]]]",
                "",
                "",
                True,
            ],
            # single segmentation mask per single bounding box
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                "",
                "[[125,240,375,425]]",
                "",
                True,
            ],
            # segmentation mask using both bounding box and input points
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                "[[[280,320]]]",
                "[[125,240,375,425]]",
                "",
                True,
            ],
            # segmentation mask using both bounding box and input points and labels
            [
                base64.encodebytes(read_image(sample_image)).decode("utf-8"),
                "[[[280,320]]]",
                "[[125,240,375,425]]",
                "[[0]]",
                True,
            ],
        ],
    }
}

# Create request json
request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
# Score the sample_score.json file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=demo_deployment.name,
    request_file=request_file_name,
)

print(f"raw response: {response}\n")

#### Visualize input bounding box and generated mask on the image

In [ ]:
# visualize sample input bounding box as prompt and output mask
import io
import base64
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(
        plt.Rectangle((x0, y0), w, h, edgecolor="green", facecolor=(0, 0, 0, 0), lw=2)
    )


def show_mask(mask, ax, random_color=False):
    if not isinstance(mask, np.ndarray):
        mask = np.array(mask)
        mask = mask > 0
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


df_from_json = pd.read_json(response)
encoded_mask = df_from_json["response"][3]["predictions"][0]["masks_per_prediction"][0][
    "encoded_binary_mask"
]
mask_iou = df_from_json["response"][3]["predictions"][0]["masks_per_prediction"][0][
    "iou_score"
]

# Load the sample image
img = Image.open(io.BytesIO(base64.b64decode(encoded_mask)))
raw_image = Image.open(sample_image).convert("RGB")

# Display the original image and bounding box
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(np.array(raw_image))
show_box([125, 240, 375, 425], axes[0])
axes[0].title.set_text(f"Input image with bounding box as prompt.")
axes[0].axis("off")

axes[1].imshow(np.array(raw_image))
show_mask(img, axes[1])
axes[1].title.set_text(f"Output mask with iou score: {mask_iou:.3f}")
axes[1].axis("off")
# Adjust the spacing between subplots
fig.tight_layout()
plt.show()

### 6. Clean up resources - delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()